This can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20230314_2023.2_updates/pipeline-orchestrators/orchestration_sdk_bigquery_houseprice_tutorial).

## Wallaroo ML Workload Orchestration House Price Model Tutorial

This tutorial provides a quick set of methods and examples regarding Wallaroo Connections and Wallaroo ML Workload Orchestration.  For full details, see the Wallaroo Documentation site.

Wallaroo provides Data Connections and ML Workload Orchestrations to provide organizations with a method of creating and managing automated tasks that can either be run on demand or a regular schedule.

| Object | Description |
|---|---|
| Orchestration | A set of instructions written as a python script with a requirements library.  Orchestrations are uploaded to the Wallaroo instance |
| Task | An implementation of an orchestration.  Tasks are run either once when requested, on a repeating schedule, or as a service. |
| Connection | Definitions set by MLOps engineers that are used by other Wallaroo users for connection information to a data source.  Usually paired with orchestrations. |

This tutorial will focus on using Google BigQuery as the data source.

## Tutorial Goals

The tutorial will demonstrate the following:

1. Create a Wallaroo connection to retrieving information from a Google BigQuery source table.
1. Create a Wallaroo connection to store inference results into a Google BigQuery destination table.
1. Upload Wallaroo ML Workload Orchestration that supports BigQuery connections with the connection details.
1. Run the orchestration once as a Run Once Task and verify that the inference request succeeded and the inference results were saved to the external data store.
1. Schedule the orchestration as a Scheduled Task and verify that the inference request succeeded and the inference results were saved to the external data store.

## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed.  These are included by default in a Wallaroo instance's JupyterHub service.
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * [`pyarrow`](https://pypi.org/project/pyarrow/): PyArrow for Apache Arrow support
* The following Python libraries.  These are **not** included in a Wallaroo instance's JupyterHub service.
  * [`google-cloud-bigquery`](https://pypi.org/project/google-cloud-bigquery/): Specifically for its support for Google BigQuery.
  * [`google-auth`](https://pypi.org/project/google-auth/): Used to authenticate for bigquery.
  * [`db-dtypes`](https://pypi.org/project/db-dtypes/): Converts the BigQuery results to Apache Arrow table or pandas DataFrame.

## Tutorial Resources

* Models:
  * `models/rf_model.onnx`: A model that predicts house price values.
* Data:
  * `data/xtest-1.df.json` and `data/xtest-1k.df.json`:  DataFrame JSON inference inputs with 1 input and 1,000 inputs.
  * `data/xtest-1k.arrow`:  Apache Arrow inference inputs with 1 input and 1,000 inputs.
  * Sample inference inputs in `CSV` that can be imported into Google BigQuery.
    * `data/xtest-1k.df.json`: Random sample housing prices.
    * `data/smallinputs.df.json`: Sample housing prices that return results lower than $1.5 million.
    * `data/biginputs.df.json`: Sample housing prices that return results higher than $1.5 million.
  * SQL queries to create the inputs/outputs tables with schema.
    * `./resources/create_inputs_table.sql`: Inputs table with schema.
    * `./resources/create_outputs_table.sql`: Outputs table with schema.
    * `./resources/housrpricesga_inputs.avro`: Avro container of inputs table.


## Initial Steps

For this tutorial, we'll create a workspace, upload our sample model and deploy a pipeline.  We'll perform some quick sample inferences to verify that everything it working.

### Load Libraries

Here we'll import the various libraries we'll use for the tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError, RequiredAttributeMissing

# to display dataframe tables
from IPython.display import display
# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pyarrow as pa

import os
# Used for the Wallaroo SDK version 2023.1
os.environ["ARROW_ENABLED"]="True"

import time
import json

# for Big Query connections
from google.cloud import bigquery
from google.oauth2 import service_account
import db_dtypes

In [2]:
wallaroo.__version__

'2023.2.0rc2'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

# # SSO login through keycloak

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

Please log into the following URL in a web browser:

	https://product-uat-ee.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=LVKS-MMUE

Login successful!


## Variable Declaration

The following variables will be used for our big query testing.  

We'll use two connections:

* bigquery_input_connection: The connection that will draw inference input data from a BigQuery table.
* bigquery_output_connection:  The connection that will upload inference results into a BigQuery table.

Not that for the connection arguments, we'll retrieve the information from the files `./bigquery_service_account_input_key.json` and `./bigquery_service_account_output_key.json` that include the  [service account key file(SAK)](https://cloud.google.com/bigquery/docs/authentication/service-account-file) information, as well as the dataset and table used.

| Field | Included in SAK | 
|---|---|
| type | √ | 
| project_id | √ |
| private_key_id | √ |
| private_key | √ |
| client_email | √ |
| auth_uri | √ |
| token_uri | √ |
| auth_provider_x509_cert_url | √ |
| client_x509_cert_url | √ |
| database | 🚫 |
| table | 🚫 |


In [16]:
# Setting variables for later steps

workspace_name = 'bigqueryworkspace'
pipeline_name = 'bigquerypipeline'
model_name = 'bigquerymodel'
model_file_name = './models/rf_model.onnx'

bigquery_connection_input_name = "bigqueryhouseinputs"
bigquery_connection_input_type = "BIGQUERY"
bigquery_connection_input_argument = json.load(open('./bigquery_service_account_input_key.json.example'))

bigquery_connection_output_name = "bigqueryhouseoutputs"
bigquery_connection_output_type = "BIGQUERY"
bigquery_connection_output_argument = json.load(open('./bigquery_service_account_output_key.json.example'))

### Helper Methods

The following helper methods are used to either create or get workspaces, pipelines, and connections.

In [5]:
# helper methods to retrieve workspaces and pipelines

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

def get_connection(name, connection_type, connection_arguments):
    try:
        connection = wl.get_connection(name)
    except RequiredAttributeMissing:
        connection =wl.create_connection(name, 
                  connection_type, 
                  connection_arguments)
    return connection


### Create the Workspace and Pipeline

We'll now create our workspace and pipeline for the tutorial.  If this tutorial has been run previously, then this will retrieve the existing ones with the assumption they're for us with this tutorial.

We'll set the retrieved workspace as the current workspace in the SDK, so all commands will default to that workspace.

In [6]:
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload the Model and Deploy Pipeline

We'll upload our model into our sample workspace, then add it as a pipeline step before deploying the pipeline to it's ready to accept inference requests.

In [7]:
# Upload the model

housing_model_control = wl.upload_model(model_name, model_file_name).configure()

# Add the model as a pipeline step

pipeline.add_model_step(housing_model_control)

name,bigquerypipeline
created,2023-05-10 15:50:30.828463+00:00
last_updated,2023-05-10 20:50:15.732156+00:00
deployed,True
tags,
versions,"af4ead91-53d9-480b-8240-4d5e05e8d509, c69695c5-695b-4f6a-a021-786a807ec69d, c3713baa-c9d4-4cb1-8e01-54b4d7634224"
steps,bigquerymodel


In [8]:
#deploy the pipeline
pipeline.deploy()

name,bigquerypipeline
created,2023-05-10 15:50:30.828463+00:00
last_updated,2023-05-10 22:07:33.391328+00:00
deployed,True
tags,
versions,"ed537c22-9825-4c03-904e-747041daf8ba, af4ead91-53d9-480b-8240-4d5e05e8d509, c69695c5-695b-4f6a-a021-786a807ec69d, c3713baa-c9d4-4cb1-8e01-54b4d7634224"
steps,bigquerymodel


## Create Connections

We will create the data source connection via the Wallaroo client command `create_connection`.

Connections are created with the Wallaroo client command [`create_connection`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/#create-orchestration) with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the connection. This must be unique - **if submitting the name of an existing** connection it will return an error. |
| **type** | string (Required) | The user defined type of connection. |
| **details** | Dict (Required) | User defined configuration details for the data connection.  These can be `{'username':'dataperson', 'password':'datapassword', 'port': 3339}`, or `{'token':'abcde123==', 'host':'example.com', 'port:1234'}`, or other user defined combinations.  |

* **IMPORTANT NOTE**:  Data connections names **must** be unique.  Attempting to create a data connection with the same `name` as an existing data connection will result in an error.


In [ ]:
connection_input = get_connection(bigquery_connection_input_name, bigquery_connection_input_type, bigquery_connection_input_argument)
connection_output = get_connection(bigquery_connection_output_name, bigquery_connection_output_type, bigquery_connection_output_argument)
wl.list_connections()

### Get Connection by Name

The Wallaroo client method `get_connection(name)` retrieves the connection that matches the `name` parameter.  We'll retrieve our connection and store it as `inference_source_connection`.

In [40]:
big_query_input_connection = wl.get_connection(name=bigquery_connection_input_name)
big_query_output_connection = wl.get_connection(name=bigquery_connection_output_name)
display(big_query_input_connection)
display(big_query_output_connection)

Field,Value
Name,bigqueryhouseinputs
Connection Type,BIGQUERY
Details,*****
Created At,2023-05-10T22:14:06.851877+00:00


Field,Value
Name,bigqueryhouseoutputs
Connection Type,BIGQUERY
Details,*****
Created At,2023-05-10T22:14:07.321104+00:00


### Add Connection to Workspace

The method Workspace [`add_connection(connection_name)`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#add-data-connection-to-workspace) adds a Data Connection to a workspace, and takes the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the Data Connection |

We'll add both connections to our sample workspace, then list the connections available to the workspace to confirm.

In [42]:
workspace.add_connection(bigquery_connection_input_name)
workspace.add_connection(bigquery_connection_output_name)


workspace.list_connections()

name,connection type,details,created at
bigqueryhouseinputs,BIGQUERY,*****,2023-05-10T22:14:06.851877+00:00
bigqueryhouseoutputs,BIGQUERY,*****,2023-05-10T22:14:07.321104+00:00


## Big Query Connection Inference Example

We can test the BigQuery connection with a simple inference to our deployed pipeline.  We'll request the data, format the table into a pandas DataFrame, then submit it for an inference request.

### Create Google Credentials

From our BigQuery request, we'll create the credentials for our BigQuery connection.

In [43]:
bigquery_input_credentials = service_account.Credentials.from_service_account_info(
    big_query_input_connection.details())

bigquery_output_credentials = service_account.Credentials.from_service_account_info(
    big_query_output_connection.details())

### Connect to Google BigQuery

We can now generate a client from our connection details, specifying the project that was included in the `big_query_connection` details.

In [44]:
bigqueryinputclient = bigquery.Client(
    credentials=bigquery_input_credentials, 
    project=big_query_input_connection.details()['project_id']
)
bigqueryoutputclient = bigquery.Client(
    credentials=bigquery_output_credentials, 
    project=big_query_output_connection.details()['project_id']
)

### Query Data

Now we'll create our query and retrieve information from out dataset and table as defined in the file `bigquery_service_account_key.json`.  The table is expected to be in the format of the file `./data/xtest-1k.df.json`.

In [45]:
inference_dataframe_input = bigqueryinputclient.query(
        f"""
        SELECT tensor
        FROM {big_query_input_connection.details()['dataset']}.{big_query_input_connection.details()['table']}"""
    ).to_dataframe()

In [46]:
inference_dataframe_input.head(5)

,tensor
0,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]"
1,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]"
2,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]"
3,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]"
4,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]"


### Sample Inference

With our data retrieved, we'll perform an inference and display the results.

In [47]:
result = pipeline.infer(inference_dataframe_input)
display(result.head(5))

,time,in.tensor,out.variable,check_failures
0,2023-05-10 22:15:35.591,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-05-10 22:15:35.591,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-05-10 22:15:35.591,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-05-10 22:15:35.591,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-05-10 22:15:35.591,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0


### Upload the Results

With the query complete, we'll upload the results back to the BigQuery dataset.

In [48]:
output_table = bigqueryoutputclient.get_table(f"{big_query_output_connection.details()['dataset']}.{big_query_output_connection.details()['table']}")

bigqueryoutputclient.insert_rows_from_dataframe(
    output_table, 
    dataframe=result.rename(columns={"in.tensor":"in_tensor", "out.variable":"out_variable"})
)

[[], []]

## Wallaroo ML Workload Orchestration Example

With the pipeline deployed and our connections set, we will now generate our ML Workload Orchestration.  See the [Wallaroo ML Workload Orchestrations guide](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/) for full details.

Orchestrations are uploaded to the Wallaroo instance as a ZIP file with the following requirements:

| Parameter | Type | Description |
|---|---|---|
| **User Code** | (*Required*) Python script as `.py` files | If `main.py` exists, then that will be used as the task entrypoint. Otherwise, the **first** `main.py` found in any subdirectory will be used as the entrypoint. |
| Python Library Requirements | (*Optional*) `requirements.txt` file in the [requirements file format](https://pip.pypa.io/en/stable/reference/requirements-file-format/).  A standard Python requirements.txt for any dependencies to be provided in the task environment. The Wallaroo SDK will already be present and **should not be included in the requirements.txt**. Multiple requirements.txt files are not allowed. |
| Other artifacts | &nbsp; | Other artifacts such as files, data, or code to support the orchestration.

For our example, our orchestration will:

1. Use the `bigquery_remote_inference` to open a connection to the input and output tables.
1. Deploy the pipeline.
1. Perform an inference with the input data.
1. Save the inference results to the output table.
1. Undeploy the pipeline.

This sample script is stored in `bigquery_remote_inference/main.py` with an `requirements.txt` file having the specific libraries for the Google BigQuery connection., and packaged into the orchestration as `./bigquery_remote_inference/bigquery_remote_inference.zip`.  We'll display the steps in uploading the orchestration to the Wallaroo instance.

Note that the orchestration assumes the pipeline is already deployed.

### Upload the Orchestration

Orchestrations are uploaded with the Wallaroo client `upload_orchestration(path)` method with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **path** | string (Required) | The path to the ZIP file to be uploaded. |

Once uploaded, the deployment will be prepared and any requirements will be downloaded and installed.


For this example, the orchestration `./bigquery_remote_inference/bigquery_remote_inference.zi` will be uploaded and saved to the variable `orchestration`.

In [49]:
orchestration = wl.upload_orchestration(path="./bigquery_remote_inference/bigquery_remote_inference.zip")

### Orchestration Status

We will loop until the uploaded orchestration's `status` displays `ready`.

In [50]:
while orchestration.status() != 'ready':
    print(orchestration.status())
    time.sleep(5)

packaging
packaging
packaging
packaging
packaging
packaging
packaging


In [51]:
wl.list_orchestrations()

id,status,name,sha,created at,updated at
f5db5dd3-0961-4807-8147-ea42c013cd01,ready,bigquery_remote_inference.zip,ff31c711cf31fb0e2cd2944a8b47c19f55702698320989050f58085ba277a6a1,2023-10-May 22:15:50,2023-10-May 22:16:54
97c8a469-3a9c-4bec-842c-a5cc162d09f7,ready,bigquery_remote_inference.zip,4ee31c3787c7ee2656262eba0e7803c09358543ea45590ff8209910daa201a71,2023-10-May 20:53:48,2023-10-May 20:54:44
404e6010-fff1-4cf7-96da-b7a08e22c020,ready,bigquery_remote_inference.zip,93ff9ddbec3a56ca86e66c10a471baf1290adfead84f67ccfa7a15678b8427ca,2023-10-May 20:51:45,2023-10-May 20:52:45


## Task Management Tutorial

Once an Orchestration has the status `ready`, it can be run as a task.  Tasks have three run options.

| Type | SDK Call |  How triggered |
|---|---|:---|
| Once       | `orchestration.run_once(json_args)` | User makes one api call. Task runs once and exits.| Single batch, experimentation. |
| Scheduled  | `orchestration.run_scheduled(name, schedule, timeout, json_args)` | User provides schedule. Task runs exits whenever schedule dictates. | Recurrent batch. |

### Run Task Once

We'll do both a Run Once task and generate our Run Once Task from our orchestration.

Tasks are generated and run once with the Orchestration `run_once(arguments)` method.  Any arguments for the orchestration are passed in as a `Dict`.  If there are no arguments, then an empty set `{}` is passed.

We'll display the last 5 rows of our BigQuery output table, then start the task that will perform the same inference we did above.

In [52]:
task_inference_results = bigqueryoutputclient.query(
        f"""
        SELECT *
        FROM {big_query_output_connection.details()['dataset']}.{big_query_output_connection.details()['table']}
        ORDER BY time DESC
        LIMIT 5
        """
    ).to_dataframe()

display(task_inference_results)

,time,in_tensor,out_variable,check_failures
11995,2023-05-10 21:26:08.181,"[2.0, 1.0, 1010.0, 6000.0, 1.0, 0.0, 0.0, 4.0, 6.0, 1010.0, 0.0, 47.771, -122.353, 1610.0, 7313.0, 70.0, 0.0, 0.0]",[340764.53],0
11996,2023-05-10 21:26:08.181,"[2.0, 1.0, 700.0, 8100.0, 1.0, 0.0, 0.0, 3.0, 6.0, 700.0, 0.0, 47.7492, -122.311, 1230.0, 8100.0, 65.0, 0.0, 0.0]",[340764.53],0
11997,2023-05-10 21:26:08.181,"[3.0, 1.5, 1120.0, 6653.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1120.0, 0.0, 47.7321, -122.334, 1580.0, 7355.0, 78.0, 0.0, 0.0]",[340764.53],0
11998,2023-05-10 21:26:08.181,"[2.0, 1.0, 1170.0, 7142.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1170.0, 0.0, 47.7497, -122.313, 1170.0, 7615.0, 63.0, 0.0, 0.0]",[340764.53],0
11999,2023-05-10 21:26:08.181,"[3.0, 1.75, 1300.0, 10030.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1300.0, 0.0, 47.7359, -122.192, 1520.0, 7713.0, 48.0, 0.0, 0.0]",[340764.53],0


In [53]:
# Example: run once

import datetime
task_start = datetime.datetime.now()

task = orchestration.run_once({})
task

Field,Value
ID,8defc235-1c71-4337-89c6-1fe84eedef15
Status,pending
Type,Temporary Run
Created At,2023-10-May 22:17:13
Updated At,2023-10-May 22:17:13


### Task Status

The list of tasks in the Wallaroo instance is retrieves through the Wallaroo Client `list_tasks()` method.  This returns an array list of the following.

| Parameter | Type | Description |
| --- | --- | ---|
| **id** | string | The UUID identifier for the task. |
| **status** | enum | The status of the task.  Values are: <br><ul><li>`pending`: The task has not been started.</li><li>`started`: The task has been scheduled to execute.</li><li>`pending_kill`: The task kill command has been issued and the task is scheduled to be stopped.</li></ul> |
| **type** | string | The type of the task.  Values are: <br><ul><li>`Temporary Run`: The task runs once then stop.</li><li>`Scheduled Run`: The task repeats on a `cron` like schedule.</li></ul> |
| **created at** | DateTime | The date and time the task was started. |
| **updated at** | DateTime | The date and time the task was updated. |

For this example, the status of the previously created task will be generated, then looped until it has reached status `started`.

In [54]:
while task.status() != "started":
    display(task.status())
    time.sleep(5)

'started'

In [55]:
wl.list_tasks()

id,status,type,created at,updated at
c266e5ff-f485-4caa-a1aa-48586f2e1cbf,started,Temporary Run,2023-10-May 20:54:55,2023-10-May 20:55:01
00b07dc2-1070-4497-9cff-ed6106b28dc0,started,Temporary Run,2023-10-May 21:05:40,2023-10-May 21:05:45
8defc235-1c71-4337-89c6-1fe84eedef15,started,Temporary Run,2023-10-May 22:17:13,2023-10-May 22:17:20


### Task Results

We can view the inferences from our logs and verify that new entries were added from our task.  We'll query the last 5 rows of our inference output table after a wait of 60 seconds.

In [56]:
time.sleep(60)

task_inference_results = bigqueryoutputclient.query(
        f"""
        SELECT *
        FROM {big_query_output_connection.details()['dataset']}.{big_query_output_connection.details()['table']}
        ORDER BY time DESC LIMIT 5"""
    ).to_dataframe()

display(task_inference_results)

,time,in_tensor,out_variable,check_failures
12995,2023-05-10 21:26:08.181,"[2.0, 1.0, 1010.0, 6000.0, 1.0, 0.0, 0.0, 4.0, 6.0, 1010.0, 0.0, 47.771, -122.353, 1610.0, 7313.0, 70.0, 0.0, 0.0]",[340764.53],0
12996,2023-05-10 21:26:08.181,"[2.0, 1.0, 700.0, 8100.0, 1.0, 0.0, 0.0, 3.0, 6.0, 700.0, 0.0, 47.7492, -122.311, 1230.0, 8100.0, 65.0, 0.0, 0.0]",[340764.53],0
12997,2023-05-10 21:26:08.181,"[3.0, 1.5, 1120.0, 6653.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1120.0, 0.0, 47.7321, -122.334, 1580.0, 7355.0, 78.0, 0.0, 0.0]",[340764.53],0
12998,2023-05-10 21:26:08.181,"[2.0, 1.0, 1170.0, 7142.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1170.0, 0.0, 47.7497, -122.313, 1170.0, 7615.0, 63.0, 0.0, 0.0]",[340764.53],0
12999,2023-05-10 21:26:08.181,"[3.0, 1.75, 1300.0, 10030.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1300.0, 0.0, 47.7359, -122.192, 1520.0, 7713.0, 48.0, 0.0, 0.0]",[340764.53],0


## Scheduled Run Task Example

The other method of using tasks is as a **scheduled run** through the Orchestration `run_scheduled(name, schedule, timeout, json_args)`.  This sets up a task to run on an regular schedule as defined by the `schedule` parameter in the `cron` service format.  For example:

```python
schedule={'42 * * * *'}
```

Runs on the 42nd minute of every hour.

For our example, we will create a scheduled task to run every 1 minute, display the inference results, then use the Orchestration `kill` task to keep the task from running any further.

In [57]:
task_inference_results = bigqueryoutputclient.query(
        f"""
        SELECT *
        FROM {big_query_output_connection.details()['dataset']}.{big_query_output_connection.details()['table']}
        ORDER BY time DESC LIMIT 5"""
    ).to_dataframe()

display(task_inference_results.tail(5))

scheduled_task = orchestration.run_scheduled(name="simple_inference_schedule", schedule="*/1 * * * *", timeout=120, json_args={})

,time,in_tensor,out_variable,check_failures
12995,2023-05-10 21:26:08.181,"[2.0, 1.0, 1010.0, 6000.0, 1.0, 0.0, 0.0, 4.0, 6.0, 1010.0, 0.0, 47.771, -122.353, 1610.0, 7313.0, 70.0, 0.0, 0.0]",[340764.53],0
12996,2023-05-10 21:26:08.181,"[2.0, 1.0, 700.0, 8100.0, 1.0, 0.0, 0.0, 3.0, 6.0, 700.0, 0.0, 47.7492, -122.311, 1230.0, 8100.0, 65.0, 0.0, 0.0]",[340764.53],0
12997,2023-05-10 21:26:08.181,"[3.0, 1.5, 1120.0, 6653.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1120.0, 0.0, 47.7321, -122.334, 1580.0, 7355.0, 78.0, 0.0, 0.0]",[340764.53],0
12998,2023-05-10 21:26:08.181,"[2.0, 1.0, 1170.0, 7142.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1170.0, 0.0, 47.7497, -122.313, 1170.0, 7615.0, 63.0, 0.0, 0.0]",[340764.53],0
12999,2023-05-10 21:26:08.181,"[3.0, 1.75, 1300.0, 10030.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1300.0, 0.0, 47.7359, -122.192, 1520.0, 7713.0, 48.0, 0.0, 0.0]",[340764.53],0


In [58]:
while scheduled_task.status() != "started":
    display(scheduled_task.status())
    time.sleep(5)

In [59]:
#wait 120 seconds to give the scheduled event time to finish
time.sleep(120)
task_inference_results = bigqueryoutputclient.query(
        f"""
        SELECT *
        FROM {big_query_output_connection.details()['dataset']}.{big_query_output_connection.details()['table']}
        ORDER BY time DESC LIMIT 5"""
    ).to_dataframe()

display(task_inference_results.tail(5))

,time,in_tensor,out_variable,check_failures
14995,2023-05-10 21:26:08.181,"[2.0, 1.0, 1010.0, 6000.0, 1.0, 0.0, 0.0, 4.0, 6.0, 1010.0, 0.0, 47.771, -122.353, 1610.0, 7313.0, 70.0, 0.0, 0.0]",[340764.53],0
14996,2023-05-10 21:26:08.181,"[2.0, 1.0, 700.0, 8100.0, 1.0, 0.0, 0.0, 3.0, 6.0, 700.0, 0.0, 47.7492, -122.311, 1230.0, 8100.0, 65.0, 0.0, 0.0]",[340764.53],0
14997,2023-05-10 21:26:08.181,"[3.0, 1.5, 1120.0, 6653.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1120.0, 0.0, 47.7321, -122.334, 1580.0, 7355.0, 78.0, 0.0, 0.0]",[340764.53],0
14998,2023-05-10 21:26:08.181,"[2.0, 1.0, 1170.0, 7142.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1170.0, 0.0, 47.7497, -122.313, 1170.0, 7615.0, 63.0, 0.0, 0.0]",[340764.53],0
14999,2023-05-10 21:26:08.181,"[3.0, 1.75, 1300.0, 10030.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1300.0, 0.0, 47.7359, -122.192, 1520.0, 7713.0, 48.0, 0.0, 0.0]",[340764.53],0


## Kill Task

With our testing complete, we will kill the scheduled task so it will not run again.  First we'll show all the tasks to verify that our task is there, then issue it the kill command.

In [60]:
scheduled_task.kill()

<ArbexStatus.PENDING_KILL: 'pending_kill'>

## Cleanup

With the tutorial complete, we can undeploy the pipeline and return the resources back to the Wallaroo instance.

In [61]:
pipeline.undeploy()

name,bigquerypipeline
created,2023-05-10 15:50:30.828463+00:00
last_updated,2023-05-10 22:07:33.391328+00:00
deployed,False
tags,
versions,"ed537c22-9825-4c03-904e-747041daf8ba, af4ead91-53d9-480b-8240-4d5e05e8d509, c69695c5-695b-4f6a-a021-786a807ec69d, c3713baa-c9d4-4cb1-8e01-54b4d7634224"
steps,bigquerymodel
